In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pydantic import BaseModel, Field
from typing import Dict, Optional
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.tools import TavilySearchResults

In [3]:
class State(BaseModel):
    company_name: str = Field(description="회사 이름")
    segment: Dict[str, dict] = Field(description="사업부 정보", default_factory=dict)

In [4]:
# tavily_search_tool = TavilySearchResults(
#     max_results=1,
#     search_depth="advanced",
#     include_answer=False,
#     include_raw_content=True,
#     include_images=False,
#     include_domains=["n.news.naver.com"],
#     # exclude_domains=[...],
#     # name="...",            # overwrite default tool name
#     #description="search action을 수행합니다.",     # overwrite default tool description
#     # args_schema=...,       # overwrite default args_schema: BaseModel
# )

In [5]:
state = State(company_name="네이버")

In [6]:
import pandas as pd

naver_income_stmt_cum = pd.read_csv("naver_segment_sales+income_stmt_cum.csv",header=0)
naver_income_stmt_cum

,계정,2024-3Q,2024-2Q,2024-1Q,2023-4Q,2023-3Q
0,서치플랫폼,2.881470e+12,1.883760e+12,9.053540e+11,3.589060e+12,2.660720e+12
1,커머스,2.147880e+12,1.422500e+12,7.034510e+11,2.546650e+12,1.886200e+12
2,핀테크,1.107490e+12,7.223740e+11,3.539100e+11,1.354770e+12,9.987490e+11
3,콘텐츠,1.329070e+12,8.663160e+11,4.463190e+11,1.732980e+12,1.266640e+12
4,클라우드,3.861910e+11,2.415980e+11,1.170200e+11,4.471840e+11,3.213300e+11
5,영업수익,7.852100e+12,5.136540e+12,2.526060e+12,9.670640e+12,7.133640e+12
6,영업비용,6.414850e+12,4.224560e+12,2.086760e+12,8.181820e+12,6.050310e+12
7,영업이익,1.437250e+12,9.119780e+11,4.392930e+11,1.488820e+12,1.083330e+12
8,영업이익률,1.830400e-01,1.775471e-01,1.739048e-01,1.539526e-01,1.518621e-01
9,순이익,1.417940e+12,8.878600e+11,5.558030e+11,9.850180e+11,6.865770e+11


In [7]:
segments = []
for item in naver_income_stmt_cum.iloc[:, 0]:
    if item == '영업수익':
        break
    segments.append(item)

segments

['서치플랫폼', '커머스', '핀테크', '콘텐츠', '클라우드']

In [8]:
# YoY 계산 ((현재 - 이전) / 이전) * 100
yoy = ((naver_income_stmt_cum['2024-3Q'] - naver_income_stmt_cum['2023-3Q']) / naver_income_stmt_cum['2023-3Q']) * 100

# 각 사업부별 YoY와 사업부 이름을 함께 보여주기
for segment, growth in zip(naver_income_stmt_cum['계정'], yoy):
    if segment in segments:  # segments 리스트에 있는 사업부만 출력
        state.segment[segment] = {"yoy":growth}
        print(f"{segment}: {growth:.2f}%")

서치플랫폼: 8.30%
커머스: 13.87%
핀테크: 10.89%
콘텐츠: 4.93%
클라우드: 20.19%


In [9]:
for segment, growth in zip(naver_income_stmt_cum['계정'], yoy):
    if segment == '영업수익':  # segments 리스트에 있는 사업부만 출력
        state.segment['영업수익'] = {"yoy":growth}
        print(f"{segment}: {growth:.2f}%")

영업수익: 10.07%


In [10]:
for segment, sales in zip(naver_income_stmt_cum['계정'], naver_income_stmt_cum['2024-3Q']):
    if segment in segments:
        state.segment[segment].update({"sales":sales})

In [11]:
for segment, sales in zip(naver_income_stmt_cum['계정'], naver_income_stmt_cum['2024-3Q']):
    if segment == '영업수익':  # segments 리스트에 있는 사업부만 출력
        state.segment['영업수익'].update({"sales":sales})

In [12]:
state.segment

{'서치플랫폼': {'yoy': 8.296626477043807, 'sales': 2881470000000.0},
 '커머스': {'yoy': 13.873396246421377, 'sales': 2147880000000.0},
 '핀테크': {'yoy': 10.88772053839353, 'sales': 1107490000000.0},
 '콘텐츠': {'yoy': 4.928787974483673, 'sales': 1329070000000.0},
 '클라우드': {'yoy': 20.18516789593253, 'sales': 386191000000.0},
 '영업수익': {'yoy': 10.071436181248284, 'sales': 7852100000000.0}}

In [11]:
# for segment in segments:
#     news_result = tavily_search_tool.invoke(
#         {
#             "query": f"네이버의 {segment} 사업부의 매출 영향"
#         }
#     )
#     State['segment'][segment] = {"news_result":news_result}
# State

In [13]:
duckduckgo_search_tool = DuckDuckGoSearchResults(max_results=1,backend="news")

In [14]:
import time
for segment in segments:
    news_result = duckduckgo_search_tool.invoke(
        {
            "query": f"네이버 {segment} 사업부 매출"
        }
    )
    state.segment[segment].update({"news_result":news_result})
    # time.sleep(1)
state

State(company_name='네이버', segment={'서치플랫폼': {'yoy': 8.296626477043807, 'sales': 2881470000000.0, 'news_result': 'snippet: 네이버가 최수연 대표 취임 3년차인 올해 매출 10조원을 ... 최대치를 기록했다. 네이버 광고 사업 성과가 포함된 서치 플랫폼 매출이 성과를 견인했다., title: 네이버, 최수연號 3년차 매출 10조 돌파 유력…\'플랫폼 역량 강화\' 통했다, link: https://www.etnews.com/20241108000168, date: 2024-11-10T15:00:00+00:00, source: Etnews.com, snippet: 사업 부문별 매출액은 서치플랫폼 ... 네이버 측은 브랜드와의 협업과 멤버십 혜택 강화, 배송 품질 개선 등에 따른 거래액 상승과 브랜드 솔루션 패키지, 도착보장 사용률 증가가 커머스 매출 ..., title: 네이버, 3분기 영업익 사상 최대 실적…"검색 등 전 사업 고른 성장", link: https://news.mtn.co.kr/news-detail/2024110808322468489, date: 2024-11-08T08:54:00+00:00, source: MTN 뉴스, snippet: 네이버 3분기 실적. / 네이버 네이버는 3분기 매출 2조7156억원 ... 사업 부문별 매출액은 서치플랫폼 9977억원 커머스 7254억원 핀테크 3851 ..., title: 네이버 \'사상 최대\' 영업익 기록… 3분기 5253억원, link: https://it.chosun.com/news/articleView.html?idxno=2023092126781, date: 2024-11-08T09:52:00+00:00, source: 조선일보, snippet: 네이버가 홈피드, 클립 등 모바일 애플리케이션(앱) 개편 효과에 따른 광고 매출 증가로 ... 부문별 매출액은 서치 플랫폼 9977억원 커머스 ..., title:

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class yoy_prediction(BaseModel):
    """yoy 예측값을 나타내는 모델"""
    business_segment: str = Field(description="사업부")
    yoy: float = Field(description="사업부 yoy 예측값")
    reason: str = Field(description="사업부 yoy 예측값의 근거")

parser = PydanticOutputParser(pydantic_object=yoy_prediction)


template = """
다음 내용을 참고하여 {company_name}의 {business_segment} 사업부의 매출 혹은 비용이 yoy로 얼마나 변할지 예측하시오.

직전 분기 yoy : {yoy}

{company_name}의 {business_segment} 사업부의 매출 혹은 비용에 큰 영향을 미치는 뉴스 : {news}

응답 형식:
{format_instructions}
"""

for segment in segments:
    prompt = ChatPromptTemplate.from_template(template=template, partial_variables={"format_instructions": parser.get_format_instructions()})

    llm = ChatOpenAI(temperature=0, model="gpt-4o")

    chain = prompt | llm | parser

    result = chain.invoke({"company_name": state.company_name, "news": state.segment[segment]['news_result'], "business_segment": segment, "yoy": state.segment[segment]['yoy']})
    state.segment[segment].update({"yoy_prediction":result.yoy})
    state.segment[segment].update({"yoy_prediction_reason":result.reason})

In [17]:
# 2024-4Q 열 추가 (모든 행 0으로 초기화)
naver_income_stmt_cum['2024-4Q'] = 0

# segments에 있는 계정에 대해서만 yoy_prediction을 적용하여 업데이트
for segment in segments:
    yoy_prediction = state.segment[segment]['yoy_prediction']
    growth_rate = 1 + (yoy_prediction / 100)  # yoy_prediction을 성장률로 변환
    
    # 해당 segment 행의 2024-4Q 값을 업데이트
    naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == segment, '2024-4Q'] = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == segment, '2023-4Q'] * growth_rate

naver_income_stmt_cum

/var/folders/f7/84_7pkk50r10snsc2qr066600000gn/T/ipykernel_7337/3923098168.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.7763045e+12]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == segment, '2024-4Q'] = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == segment, '2023-4Q'] * growth_rate


,계정,2024-3Q,2024-2Q,2024-1Q,2023-4Q,2023-3Q,2024-4Q
0,서치플랫폼,2.881470e+12,1.883760e+12,9.053540e+11,3.589060e+12,2.660720e+12,3.965911e+12
1,커머스,2.147880e+12,1.422500e+12,7.034510e+11,2.546650e+12,1.886200e+12,2.941381e+12
2,핀테크,1.107490e+12,7.223740e+11,3.539100e+11,1.354770e+12,9.987490e+11,1.524116e+12
3,콘텐츠,1.329070e+12,8.663160e+11,4.463190e+11,1.732980e+12,1.266640e+12,1.776304e+12
4,클라우드,3.861910e+11,2.415980e+11,1.170200e+11,4.471840e+11,3.213300e+11,5.589800e+11
5,영업수익,7.852100e+12,5.136540e+12,2.526060e+12,9.670640e+12,7.133640e+12,0.000000e+00
6,영업비용,6.414850e+12,4.224560e+12,2.086760e+12,8.181820e+12,6.050310e+12,0.000000e+00
7,영업이익,1.437250e+12,9.119780e+11,4.392930e+11,1.488820e+12,1.083330e+12,0.000000e+00
8,영업이익률,1.830400e-01,1.775471e-01,1.739048e-01,1.539526e-01,1.518621e-01,0.000000e+00
9,순이익,1.417940e+12,8.878600e+11,5.558030e+11,9.850180e+11,6.865770e+11,0.000000e+00


In [18]:
# 2024-4Q의 segments 행들의 합계 계산
revenue_sum = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'].isin(segments), '2024-4Q'].sum()

# 영업수익 행의 2024-4Q 값을 업데이트
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', '2024-4Q'] = revenue_sum
naver_income_stmt_cum

,계정,2024-3Q,2024-2Q,2024-1Q,2023-4Q,2023-3Q,2024-4Q
0,서치플랫폼,2.881470e+12,1.883760e+12,9.053540e+11,3.589060e+12,2.660720e+12,3.965911e+12
1,커머스,2.147880e+12,1.422500e+12,7.034510e+11,2.546650e+12,1.886200e+12,2.941381e+12
2,핀테크,1.107490e+12,7.223740e+11,3.539100e+11,1.354770e+12,9.987490e+11,1.524116e+12
3,콘텐츠,1.329070e+12,8.663160e+11,4.463190e+11,1.732980e+12,1.266640e+12,1.776304e+12
4,클라우드,3.861910e+11,2.415980e+11,1.170200e+11,4.471840e+11,3.213300e+11,5.589800e+11
5,영업수익,7.852100e+12,5.136540e+12,2.526060e+12,9.670640e+12,7.133640e+12,1.076669e+13
6,영업비용,6.414850e+12,4.224560e+12,2.086760e+12,8.181820e+12,6.050310e+12,0.000000e+00
7,영업이익,1.437250e+12,9.119780e+11,4.392930e+11,1.488820e+12,1.083330e+12,0.000000e+00
8,영업이익률,1.830400e-01,1.775471e-01,1.739048e-01,1.539526e-01,1.518621e-01,0.000000e+00
9,순이익,1.417940e+12,8.878600e+11,5.558030e+11,9.850180e+11,6.865770e+11,0.000000e+00


In [19]:
# 2023-4Q의 영업수익 대비 영업비용 비율 계산
revenue_2023_4Q = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', '2023-4Q'].values[0]
cost_2023_4Q = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업비용', '2023-4Q'].values[0]
cost_ratio = cost_2023_4Q / revenue_2023_4Q

# 2024-4Q 영업비용 계산
revenue_2024_4Q = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', '2024-4Q'].values[0]
cost_2024_4Q = revenue_2024_4Q * cost_ratio

# 영업비용 행의 2024-4Q 값을 업데이트
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업비용', '2024-4Q'] = cost_2024_4Q

naver_income_stmt_cum

,계정,2024-3Q,2024-2Q,2024-1Q,2023-4Q,2023-3Q,2024-4Q
0,서치플랫폼,2.881470e+12,1.883760e+12,9.053540e+11,3.589060e+12,2.660720e+12,3.965911e+12
1,커머스,2.147880e+12,1.422500e+12,7.034510e+11,2.546650e+12,1.886200e+12,2.941381e+12
2,핀테크,1.107490e+12,7.223740e+11,3.539100e+11,1.354770e+12,9.987490e+11,1.524116e+12
3,콘텐츠,1.329070e+12,8.663160e+11,4.463190e+11,1.732980e+12,1.266640e+12,1.776304e+12
4,클라우드,3.861910e+11,2.415980e+11,1.170200e+11,4.471840e+11,3.213300e+11,5.589800e+11
5,영업수익,7.852100e+12,5.136540e+12,2.526060e+12,9.670640e+12,7.133640e+12,1.076669e+13
6,영업비용,6.414850e+12,4.224560e+12,2.086760e+12,8.181820e+12,6.050310e+12,9.109133e+12
7,영업이익,1.437250e+12,9.119780e+11,4.392930e+11,1.488820e+12,1.083330e+12,0.000000e+00
8,영업이익률,1.830400e-01,1.775471e-01,1.739048e-01,1.539526e-01,1.518621e-01,0.000000e+00
9,순이익,1.417940e+12,8.878600e+11,5.558030e+11,9.850180e+11,6.865770e+11,0.000000e+00


In [20]:
# 2024-4Q의 영업이익 계산 (영업수익 - 영업비용)
revenue_2024_4Q = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', '2024-4Q'].values[0]
cost_2024_4Q = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업비용', '2024-4Q'].values[0]
operating_profit_2024_4Q = revenue_2024_4Q - cost_2024_4Q

# 영업이익 행의 2024-4Q 값을 업데이트
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업이익', '2024-4Q'] = operating_profit_2024_4Q

naver_income_stmt_cum

,계정,2024-3Q,2024-2Q,2024-1Q,2023-4Q,2023-3Q,2024-4Q
0,서치플랫폼,2.881470e+12,1.883760e+12,9.053540e+11,3.589060e+12,2.660720e+12,3.965911e+12
1,커머스,2.147880e+12,1.422500e+12,7.034510e+11,2.546650e+12,1.886200e+12,2.941381e+12
2,핀테크,1.107490e+12,7.223740e+11,3.539100e+11,1.354770e+12,9.987490e+11,1.524116e+12
3,콘텐츠,1.329070e+12,8.663160e+11,4.463190e+11,1.732980e+12,1.266640e+12,1.776304e+12
4,클라우드,3.861910e+11,2.415980e+11,1.170200e+11,4.471840e+11,3.213300e+11,5.589800e+11
5,영업수익,7.852100e+12,5.136540e+12,2.526060e+12,9.670640e+12,7.133640e+12,1.076669e+13
6,영업비용,6.414850e+12,4.224560e+12,2.086760e+12,8.181820e+12,6.050310e+12,9.109133e+12
7,영업이익,1.437250e+12,9.119780e+11,4.392930e+11,1.488820e+12,1.083330e+12,1.657560e+12
8,영업이익률,1.830400e-01,1.775471e-01,1.739048e-01,1.539526e-01,1.518621e-01,0.000000e+00
9,순이익,1.417940e+12,8.878600e+11,5.558030e+11,9.850180e+11,6.865770e+11,0.000000e+00


In [21]:
# 2024-4Q의 영업이익률 계산 (영업이익/영업수익 * 100)
revenue_2024_4Q = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', '2024-4Q'].values[0]
operating_profit_2024_4Q = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업이익', '2024-4Q'].values[0]
operating_margin_2024_4Q = (operating_profit_2024_4Q / revenue_2024_4Q)

# 영업이익률 행의 2024-4Q 값을 업데이트
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업이익률', '2024-4Q'] = operating_margin_2024_4Q

naver_income_stmt_cum


,계정,2024-3Q,2024-2Q,2024-1Q,2023-4Q,2023-3Q,2024-4Q
0,서치플랫폼,2.881470e+12,1.883760e+12,9.053540e+11,3.589060e+12,2.660720e+12,3.965911e+12
1,커머스,2.147880e+12,1.422500e+12,7.034510e+11,2.546650e+12,1.886200e+12,2.941381e+12
2,핀테크,1.107490e+12,7.223740e+11,3.539100e+11,1.354770e+12,9.987490e+11,1.524116e+12
3,콘텐츠,1.329070e+12,8.663160e+11,4.463190e+11,1.732980e+12,1.266640e+12,1.776304e+12
4,클라우드,3.861910e+11,2.415980e+11,1.170200e+11,4.471840e+11,3.213300e+11,5.589800e+11
5,영업수익,7.852100e+12,5.136540e+12,2.526060e+12,9.670640e+12,7.133640e+12,1.076669e+13
6,영업비용,6.414850e+12,4.224560e+12,2.086760e+12,8.181820e+12,6.050310e+12,9.109133e+12
7,영업이익,1.437250e+12,9.119780e+11,4.392930e+11,1.488820e+12,1.083330e+12,1.657560e+12
8,영업이익률,1.830400e-01,1.775471e-01,1.739048e-01,1.539526e-01,1.518621e-01,1.539526e-01
9,순이익,1.417940e+12,8.878600e+11,5.558030e+11,9.850180e+11,6.865770e+11,0.000000e+00


In [22]:
# 2023-4Q의 순이익률 계산
revenue_2023_4Q = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', '2023-4Q'].values[0]
net_profit_2023_4Q = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '순이익', '2023-4Q'].values[0]
net_margin_2023_4Q = net_profit_2023_4Q / revenue_2023_4Q

# 2024-4Q의 순이익 계산
revenue_2024_4Q = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', '2024-4Q'].values[0]
net_profit_2024_4Q = revenue_2024_4Q * net_margin_2023_4Q

# 순이익과 순이익률 행 업데이트
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '순이익', '2024-4Q'] = net_profit_2024_4Q
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '순이익률', '2024-4Q'] = net_margin_2023_4Q

naver_income_stmt_cum

,계정,2024-3Q,2024-2Q,2024-1Q,2023-4Q,2023-3Q,2024-4Q
0,서치플랫폼,2.881470e+12,1.883760e+12,9.053540e+11,3.589060e+12,2.660720e+12,3.965911e+12
1,커머스,2.147880e+12,1.422500e+12,7.034510e+11,2.546650e+12,1.886200e+12,2.941381e+12
2,핀테크,1.107490e+12,7.223740e+11,3.539100e+11,1.354770e+12,9.987490e+11,1.524116e+12
3,콘텐츠,1.329070e+12,8.663160e+11,4.463190e+11,1.732980e+12,1.266640e+12,1.776304e+12
4,클라우드,3.861910e+11,2.415980e+11,1.170200e+11,4.471840e+11,3.213300e+11,5.589800e+11
5,영업수익,7.852100e+12,5.136540e+12,2.526060e+12,9.670640e+12,7.133640e+12,1.076669e+13
6,영업비용,6.414850e+12,4.224560e+12,2.086760e+12,8.181820e+12,6.050310e+12,9.109133e+12
7,영업이익,1.437250e+12,9.119780e+11,4.392930e+11,1.488820e+12,1.083330e+12,1.657560e+12
8,영업이익률,1.830400e-01,1.775471e-01,1.739048e-01,1.539526e-01,1.518621e-01,1.539526e-01
9,순이익,1.417940e+12,8.878600e+11,5.558030e+11,9.850180e+11,6.865770e+11,1.096658e+12


In [23]:
def naver_sales_earning_surprise() -> float:
    """네이버 증권 -> 종목분석 -> 컨센서스 -> 어닝서프라이즈(매출액) -> 분기
        return :
        직전 분기 매출액에 대한 발표직전 컨센서스 (단위: 억원) + 직전 분기 이전까지의 실제 매출액 = 직전 분기까지 누적 매출액 컨센서스
    """
    previous_quarter_sales_consensus = 26620.4 * (10 ** 8) # 크롤링 추가 필요
    before_previous_quarter_sales_cum = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', '2024-2Q'].values[0]

    return previous_quarter_sales_consensus + before_previous_quarter_sales_cum


In [24]:
previous_quarter_sales_cum_consensus = naver_sales_earning_surprise()
previous_quarter_sales_cum_consensus

7798580000000.0

In [25]:
previous_quarter_sales_cum = state.segment['영업수익']['sales']
previous_quarter_sales_cum

7852100000000.0

In [26]:
yoy_consensus = ((previous_quarter_sales_cum_consensus - naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', '2023-3Q'].values[0]) / naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', '2023-3Q'].values[0]) * 100
yoy_consensus

9.32118806107401

In [27]:
state.segment['영업수익'].update({"yoy_consensus":yoy_consensus})
state.segment['영업수익'].update({"sales_consensus":previous_quarter_sales_cum_consensus})

In [28]:
state.segment['영업수익']

{'yoy': 10.071436181248284,
 'sales': 7852100000000.0,
 'yoy_consensus': 9.32118806107401,
 'sales_consensus': 7798580000000.0}

In [31]:
def previous_quarter_review(state:State) -> State:
    
    result_of_business_segment = ""
    for segment in segments:
        result_of_business_segment += f"{segment} 사업부의 직전 분기 매출액은 {state.segment[segment]['sales']} 입니다. yoy로 {state.segment[segment]['yoy']}% 변했습니다.\n"

    result_of_total_business = f"총 매출액은 {state.segment['영업수익']['sales']} 입니다. yoy로 {state.segment['영업수익']['yoy']}% 변했습니다."
    consensus_of_total_business = f"총 매출액에 대한 컨센서스는 {state.segment['영업수익']['sales_consensus']} 였습니다. yoy로 {state.segment['영업수익']['yoy_consensus']}% 변화하는 것이 컨센서스였습니다."

    template = """
    당신은 증권사 소속 애널리스트입니다.
    다음 내용을 참고하여 직전 분기 실적에 대한 리뷰를 작성하시오.

    직전 분기 사업부별 매출 실적 : {result_of_business_segment}
    직전 분기 총 매출 실적 : {result_of_total_business}
    직전 분기 총 매출 실적에 대한 컨센서스 : {consensus_of_total_business}

    """
    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    chain = prompt | llm 
    review = chain.invoke({"result_of_business_segment":result_of_business_segment, "result_of_total_business":result_of_total_business, "consensus_of_total_business":consensus_of_total_business})
    state.segment['영업수익'].update({"review":review.content})

    return state.segment['영업수익']['review']

previous_quarter_review(state)

'직전 분기 실적 리뷰:\n\n이번 분기 실적은 전반적으로 긍정적인 성과를 보였습니다. 총 매출액은 7조 8,521억 원으로, 전년 동기 대비 10.07% 증가하며 시장 컨센서스를 상회했습니다. 컨센서스는 7조 7,985억 원으로, yoy 9.32% 증가를 예상했으나, 실제 실적은 이를 초과하는 성과를 기록했습니다.\n\n사업부별로 살펴보면, 서치플랫폼 사업부는 2조 8,814억 원의 매출을 기록하며 전년 동기 대비 8.30% 증가했습니다. 이는 안정적인 성장세를 유지하고 있음을 보여줍니다. 커머스 사업부는 2조 1,478억 원의 매출로, yoy 13.87% 증가하며 가장 높은 성장률을 기록했습니다. 이는 온라인 쇼핑 및 전자상거래의 지속적인 확장에 기인한 것으로 보입니다.\n\n핀테크 사업부는 1조 1,074억 원의 매출을 기록하며 전년 대비 10.89% 증가했습니다. 이는 디지털 금융 서비스의 확장과 사용자 증가에 따른 결과로 해석됩니다. 콘텐츠 사업부는 1조 3,290억 원의 매출로, yoy 4.93% 증가했습니다. 이는 콘텐츠 소비 증가와 플랫폼 확장에 따른 안정적인 성장으로 판단됩니다.\n\n마지막으로, 클라우드 사업부는 3,861억 원의 매출을 기록하며 전년 대비 20.19% 증가했습니다. 이는 클라우드 서비스 수요 증가와 기업들의 디지털 전환 가속화에 따른 결과로 보입니다.\n\n종합적으로, 이번 분기 실적은 전 사업부에서 고른 성장을 보이며, 특히 커머스와 클라우드 사업부의 두드러진 성장이 눈에 띕니다. 이러한 성과는 향후 지속적인 성장의 기반이 될 것으로 기대됩니다.'

In [32]:
state.segment['영업수익']

{'yoy': 10.071436181248284,
 'sales': 7852100000000.0,
 'yoy_consensus': 9.32118806107401,
 'sales_consensus': 7798580000000.0,
 'review': '직전 분기 실적 리뷰:\n\n이번 분기 실적은 전반적으로 긍정적인 성과를 보였습니다. 총 매출액은 7조 8,521억 원으로, 전년 동기 대비 10.07% 증가하며 시장 컨센서스를 상회했습니다. 컨센서스는 7조 7,985억 원으로, yoy 9.32% 증가를 예상했으나, 실제 실적은 이를 초과하는 성과를 기록했습니다.\n\n사업부별로 살펴보면, 서치플랫폼 사업부는 2조 8,814억 원의 매출을 기록하며 전년 동기 대비 8.30% 증가했습니다. 이는 안정적인 성장세를 유지하고 있음을 보여줍니다. 커머스 사업부는 2조 1,478억 원의 매출로, yoy 13.87% 증가하며 가장 높은 성장률을 기록했습니다. 이는 온라인 쇼핑 및 전자상거래의 지속적인 확장에 기인한 것으로 보입니다.\n\n핀테크 사업부는 1조 1,074억 원의 매출을 기록하며 전년 대비 10.89% 증가했습니다. 이는 디지털 금융 서비스의 확장과 사용자 증가에 따른 결과로 해석됩니다. 콘텐츠 사업부는 1조 3,290억 원의 매출로, yoy 4.93% 증가했습니다. 이는 콘텐츠 소비 증가와 플랫폼 확장에 따른 안정적인 성장으로 판단됩니다.\n\n마지막으로, 클라우드 사업부는 3,861억 원의 매출을 기록하며 전년 대비 20.19% 증가했습니다. 이는 클라우드 서비스 수요 증가와 기업들의 디지털 전환 가속화에 따른 결과로 보입니다.\n\n종합적으로, 이번 분기 실적은 전 사업부에서 고른 성장을 보이며, 특히 커머스와 클라우드 사업부의 두드러진 성장이 눈에 띕니다. 이러한 성과는 향후 지속적인 성장의 기반이 될 것으로 기대됩니다.'}

In [33]:
state.segment['서치플랫폼']

{'yoy': 8.296626477043807,
 'sales': 2881470000000.0,
 'news_result': 'snippet: 네이버가 최수연 대표 취임 3년차인 올해 매출 10조원을 ... 최대치를 기록했다. 네이버 광고 사업 성과가 포함된 서치 플랫폼 매출이 성과를 견인했다., title: 네이버, 최수연號 3년차 매출 10조 돌파 유력…\'플랫폼 역량 강화\' 통했다, link: https://www.etnews.com/20241108000168, date: 2024-11-10T15:00:00+00:00, source: Etnews.com, snippet: 사업 부문별 매출액은 서치플랫폼 ... 네이버 측은 브랜드와의 협업과 멤버십 혜택 강화, 배송 품질 개선 등에 따른 거래액 상승과 브랜드 솔루션 패키지, 도착보장 사용률 증가가 커머스 매출 ..., title: 네이버, 3분기 영업익 사상 최대 실적…"검색 등 전 사업 고른 성장", link: https://news.mtn.co.kr/news-detail/2024110808322468489, date: 2024-11-08T08:54:00+00:00, source: MTN 뉴스, snippet: 네이버 3분기 실적. / 네이버 네이버는 3분기 매출 2조7156억원 ... 사업 부문별 매출액은 서치플랫폼 9977억원 커머스 7254억원 핀테크 3851 ..., title: 네이버 \'사상 최대\' 영업익 기록… 3분기 5253억원, link: https://it.chosun.com/news/articleView.html?idxno=2023092126781, date: 2024-11-08T09:52:00+00:00, source: 조선일보, snippet: 네이버가 홈피드, 클립 등 모바일 애플리케이션(앱) 개편 효과에 따른 광고 매출 증가로 ... 부문별 매출액은 서치 플랫폼 9977억원 커머스 ..., title: 앱 개편으로 역대급 실적 쓴 네이버..."가속 성장 이어간다"(종합), l

In [ ]:
def peer_analysis(state:State) -> State:
    template = """
    당신은 증권사 소속 애널리스트입니다.
    다음 내용을 참고하여 다음 분기 매출액을 예측한 내용과 그 근거를 독자가 이해하기 쉽도록 풀어서 작성하시오.

    직전 분기 매출액 증가율 yoy : {previous_quarter_sales}
    직전 분기 매출액에 대한 컨센서스 : {previous_quarter_sales_consensus}

    """
